In [1]:
with open('Iran.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))

861126


In [2]:
print(text[:200])

﻿The Project Gutenberg eBook of The Persian Literature, Comprising The Shah Nameh, The Rubaiyat, The Divan, and The Gulistan, Volume 1
    
This ebook is for the use of anyone anywhere in the United S


In [3]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Á', 'Í', 'Ú', 'à', 'á', 'â', 'ä', 'è', 'é', 'ê', 'í', 'ï', 'ó', 'ú', 'ü', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [4]:
print(len(chars))

107


In [9]:
print(chars[58])

a


In [19]:
def encode(string) -> list:
    return [string_to_int[c] for c in string]

def decode(l) -> str:
    return ''.join([int_to_string[s] for s in l])

In [20]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

# print(encode('aghabidareh'))
encoded_aghabidareh = encode('aghabidareh')
decoded_aghabidareh = decode(encoded_aghabidareh)
print(f'encoded one: {encoded_aghabidareh}')
print(f'decoded that: {decoded_aghabidareh}')

encoded one: [58, 64, 65, 58, 59, 66, 61, 58, 75, 62, 65]
decoded that: aghabidareh
